# Traducción de Español a Inglés con KerasNLP

**Autoras:** Alina Rojas y Mar Iborra<br>
**Basado en el trabajo original de:** [Abheesht Sharma](https://github.com/abheesht17/)<br>
**Fecha de creación de la adaptación:** 2023/12/31<br>
**Última modificación:** 2024/01/12<br>
**Descripción:** Utilizar KerasNLP para entrenar un modelo Transformer de secuencia a secuencia en la tarea de traducción automática.

## Introducción

En este proyecto utilizaremos las capas proporcionadas por KerasNLP para construir un modelo Transformer codificador-decodificador y entrenarlo en la tarea de traducción automática del español al inglés. 

Este proyecto se basa en el ejemplo de traducción automática de inglés a español que se encuentra en la [documentación oficial de Keras](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/), creado por [fchollet](https://twitter.com/fchollet), ejemplo adapatado también a la traducción de texto español-inglés. El ejemplo original de fchollet es más detallado y utiliza capas implementadas desde cero, mientras que en este proyecto utilizaremos las capas proporcionadas por KerasNLP, lo que facilitará la implementación de tareas más avanzadas, como la tokenización de subpalabras y el cálculo de métricas para evaluar la calidad de las traducciones generadas.

Este proyecto implica:

- Tokenizar texto utilizando `keras_nlp.tokenizers.WordPieceTokenizer`.
- Implementar un modelo Transformer secuencia a secuencia utilizando las capas `keras_nlp.layers.TransformerEncoder`, `keras_nlp.layers.TransformerDecoder` y `keras_nlp.layers.TokenAndPositionEmbedding` de KerasNLP, y entrenarlo.
- Utilizar `keras_nlp.samplers` para generar traducciones de oraciones de entrada no vistas utilizando la estrategia de decodificación "top-p".


### Capas KerasNLP

Las capas KerasNLP son componentes esenciales proporcionados por la biblioteca KerasNLP que simplifican la construcción de modelos de procesamiento del lenguaje natural (NLP). A diferencia de implementar estas capas desde cero, KerasNLP ofrece una interfaz de alto nivel que facilita la creación y entrenamiento de modelos NLP avanzados. Algunas de las capas clave incluyen:

1. **WordPieceTokenizer**: Esta capa se utiliza para tokenizar el texto en subpalabras, lo que es especialmente útil para lidiar con palabras compuestas y idiomas con una gran cantidad de inflexiones. Simplifica la tokenización de palabras en fragmentos más pequeños.

2. **TransformerEncoder**: Esta capa implementa la parte codificadora de un modelo Transformer. Transforma la secuencia de entrada en representaciones enriquecidas que capturan las relaciones entre las palabras y las subpalabras.

3. **TransformerDecoder**: Esta capa es la contraparte de la capa codificadora y se utiliza para generar secuencias de salida basadas en las representaciones generadas por la capa codificadora. Es esencial para tareas de generación de texto, como la traducción automática.

4. **TokenAndPositionEmbedding**: Esta capa combina la información de tokenización con información de posición para crear embeddings que se utilizan como entrada para el modelo Transformer. Ayuda al modelo a comprender tanto qué palabras están presentes como su ubicación en la secuencia.

La principal ventaja de utilizar estas capas KerasNLP es la simplificación del proceso de construcción de modelos NLP complejos. En comparación con el ejemplo original [aquí](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/), donde se implementan estas capas desde cero, el uso de KerasNLP ahorra tiempo y reduce la complejidad del código, permitiendo concentrarse en la lógica de la aplicación en lugar de en la implementación detallada de las capas del modelo.

Este proyecto utiliza las capas KerasNLP para implementar un modelo Transformer codificador-decodificador y entrenarlo en la tarea de traducción automática inglés-español, lo que facilita significativamente la construcción y el entrenamiento del modelo en comparación con una implementación desde cero.

## Configuración

Antes de comenzar a implementar el proyecto, importemos todas las bibliotecas que necesitamos.

In [1]:
!pip install -q --upgrade rouge-score
!pip install -q --upgrade keras-nlp
!pip install --upgrade tensorflow

In [2]:
import keras_nlp
import pathlib
import random

import keras

import tensorflow as tf
import tensorflow.data as tf_data
from tensorflow.keras import backend as K
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

Using TensorFlow backend


Nota: Hemos realizado un cambio con respecto al código original en las bibliotecas utilizadas. Hemos actualizado las bibliotecas `rouge-score`, `keras-nlp` y `tensorflow` para asegurarnos de tener las versiones más recientes y compatibles con nuestro proyecto. Estas bibliotecas son esenciales para implementar y evaluar nuestro modelo Transformer de traducción automática inglés-español.

Definamos también nuestros parámetros e hiperparámetros para el proyecto.

In [3]:
BATCH_SIZE = 64
EPOCHS = 30
MAX_SEQUENCE_LENGTH = 40
ENG_VOCAB_SIZE = 15000
SPA_VOCAB_SIZE = 15000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

## Descargando los Datos

Trabajaremos con un conjunto de datos de traducción de inglés a español proporcionado por [Anki](https://www.manythings.org/anki/). Aunque el conjunto de datos original está diseñado para la traducción de inglés a español, adaptaremos el uso de estos datos para realizar traducciones en la dirección opuesta, de español a inglés. Descarguemos el conjunto de datos:

In [4]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

## Procesando los Datos

Cada línea contiene una oración en español y su correspondiente oración en inglés. La oración en inglés es la *secuencia fuente* y la española es la *secuencia objetivo*. Antes de agregar el texto a una lista, lo convertimos a minúsculas.

In [5]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    eng = eng.lower()
    spa = spa.lower()
    text_pairs.append((spa, eng))

En este proyecto, estamos trabajando con datos de traducción de español a inglés, por lo que hemos invertido el orden de las oraciones en comparación con el conjunto de datos original, que es de inglés a español.

Así es como se ven nuestras parejas de oraciones:

In [6]:
for _ in range(5):
    print(random.choice(text_pairs))

('la cámara costará al menos 500 dólares.', 'the camera will cost at least $500.')
('casi tengo treinta años.', "i'm almost thirty years old.")
('te ves más joven que tom.', 'you look younger than tom.')
('de ser tú, yo no lo haría.', 'if i were you, i would not do it.')
('¿cuándo es el cumpleaños de tom?', "when is tom's birthday?")


Ahora, vamos a dividir las parejas de oraciones en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba.

In [7]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


## Tokenización de los Datos

Definiremos dos tokenizadores: uno para el idioma fuente (español) y otro para el idioma objetivo (inglés). Utilizaremos `keras_nlp.tokenizers.WordPieceTokenizer` para tokenizar el texto. `keras_nlp.tokenizers.WordPieceTokenizer` toma un vocabulario WordPiece y tiene funciones para tokenizar el texto y deshacer la tokenización de secuencias de tokens.

Antes de definir los dos tokenizadores, primero debemos entrenarlos en el conjunto de datos que tenemos. El algoritmo de tokenización WordPiece es un algoritmo de subword tokenization; entrenarlo en un corpus nos proporciona un vocabulario de subwords. Un tokenizador de subwords es un compromiso entre los tokenizadores de palabras (los tokenizadores de palabras necesitan vocabularios muy grandes para una buena cobertura de palabras de entrada) y los tokenizadores de caracteres (los caracteres no codifican realmente el significado como lo hacen las palabras). Afortunadamente, KerasNLP facilita mucho el entrenamiento de WordPiece en un corpus con la utilidad `keras_nlp.tokenizers.compute_word_piece_vocabulary`.

En este proyecto, utilizaremos estos tokenizadores para procesar nuestros datos de traducción español-inglés.

In [8]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab

Cada vocabulario tiene algunos tokens especiales y reservados. En nuestro caso, tenemos cuatro de estos tokens:

- `"[PAD]"` - Token de relleno. Los tokens de relleno se agregan al final de la secuencia de entrada cuando esta es más corta que la longitud máxima de secuencia.
- `"[UNK]"` - Token desconocido.
- `"[START]"` - Token que marca el comienzo de la secuencia de entrada.
- `"[END]"` - Token que marca el final de la secuencia de entrada.

Estos tokens desempeñan roles específicos en el procesamiento de texto y son parte fundamental de nuestros tokenizadores.

In [9]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

spa_samples = [text_pair[0] for text_pair in train_pairs]
spa_vocab = train_word_piece(spa_samples, SPA_VOCAB_SIZE, reserved_tokens)

eng_samples = [text_pair[1] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

Vamos a ver algunos tokens.

In [10]:
print("Spanish Tokens: ", spa_vocab[100:110])
print("English Tokens: ", eng_vocab[100:110])

Spanish Tokens:  ['mi', 'qué', 'ella', 'le', 'te', 'para', 'mary', 'las', 'más', 'al']
English Tokens:  ['know', 'him', 'there', 'go', 'they', 'her', 'has', 'will', 'time', 're']


Ahora, definamos los tokenizadores. Configuraremos los tokenizadores con los vocabularios que entrenamos previamente.

In [11]:
spa_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=spa_vocab, lowercase=False
)

eng_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)

Probemos tokenizar un ejemplo de nuestro conjunto de datos. Para verificar si el texto se ha tokenizado correctamente, también podemos deshacer la tokenización y volver al texto original.

In [12]:
spa_input_ex = text_pairs[0][0]
spa_tokens_ex = spa_tokenizer.tokenize(spa_input_ex)
print("Spanish sentence: ", spa_input_ex)
print("Tokens: ", spa_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    spa_tokenizer.detokenize(spa_tokens_ex),
)

print()

eng_output_ex = text_pairs[0][1]
eng_tokens_ex = eng_tokenizer.tokenize(eng_output_ex)
print("English sentence: ", eng_output_ex)
print("Tokens: ", eng_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    eng_tokenizer.detokenize(eng_tokens_ex),
)

Spanish sentence:  mi casa está cerca de la escuela.
Tokens:  tf.Tensor([100 124  97 376  80  84 269  15], shape=(8,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'mi casa est\xc3\xa1 cerca de la escuela .', shape=(), dtype=string)

English sentence:  my house is near the school.
Tokens:  tf.Tensor([ 80 175  69 631  65 194  12], shape=(7,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'my house is near the school .', shape=(), dtype=string)


A continuación, formatearemos nuestros conjuntos de datos.

En cada paso de entrenamiento, el modelo buscará predecir las palabras objetivo N+1 (y más allá) utilizando la oración fuente y las palabras objetivo de 0 a N.

Por lo tanto, el conjunto de datos de entrenamiento generará una tupla `(inputs, targets)`, donde:

- `inputs` es un diccionario con las claves `encoder_inputs` y `decoder_inputs`.
`encoder_inputs` es la oración fuente tokenizada y `decoder_inputs` es la oración objetivo "hasta ahora",
es decir, las palabras de 0 a N utilizadas para predecir la palabra N+1 (y más allá) en la oración objetivo.
- `target` es la oración objetivo desplazada en un paso:
proporciona las siguientes palabras en la oración objetivo, es decir, lo que el modelo intentará predecir.

Añadiremos tokens especiales, `"[START]"` y `"[END]"`, a la oración en español de entrada después de tokenizar el texto. También rellenaremos la entrada a una longitud fija. Esto se puede hacer fácilmente utilizando `keras_nlp.layers.StartEndPacker`.

In [13]:
def preprocess_batch(spa, eng):
    # Tokenizar las oraciones en español e inglés
    spa = spa_tokenizer(spa)
    eng = eng_tokenizer(eng)

    # Agregar tokens especiales ("[START]" y "[END]") a `eng` y padear.
    eng_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=eng_tokenizer.token_to_id("[START]"),
        end_value=eng_tokenizer.token_to_id("[END]"),
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Padear `spa` hasta `MAX_SEQUENCE_LENGTH`.
    spa_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=spa_tokenizer.token_to_id("[PAD]"),
    )
    spa = spa_start_end_packer(spa)

    return (
        {
            "encoder_inputs": spa,
            "decoder_inputs": eng[:, :-1],
        },
        eng[:, 1:],
    )

def make_dataset(pairs):
    spa_texts, eng_texts = zip(*pairs)
    spa_texts = list(spa_texts)
    eng_texts = list(eng_texts)
    dataset = tf_data.Dataset.from_tensor_slices((spa_texts, eng_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()

# Crear datasets de entrenamiento y validación
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Echemos un vistazo rápido a las formas de las secuencias (tenemos lotes de 64 pares y todas las secuencias tienen 40 pasos de longitud):

In [14]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 40)
inputs["decoder_inputs"].shape: (64, 40)
targets.shape: (64, 40)


Ahora pasemos a la parte emocionante: ¡definir nuestro modelo!
Primero necesitamos una capa de inserción, es decir, un vector para cada token en nuestra secuencia de entrada. Esta capa de inserción puede inicializarse de manera aleatoria. También necesitamos una capa de inserción posicional que codifique el orden de las palabras en la secuencia. La convención es agregar estas dos inserciones. KerasNLP tiene una capa `keras_nlp.layers.TokenAndPositionEmbedding` que realiza todos los pasos mencionados anteriormente por nosotros.

Nuestro modelo Transformer secuencia a secuencia consta de una capa `keras_nlp.layers.TransformerEncoder` y una capa `keras_nlp.layers.TransformerDecoder` encadenadas.

La secuencia de origen se pasará a `keras_nlp.layers.TransformerEncoder`, que producirá una nueva representación de la misma. Esta nueva representación se pasará luego al `keras_nlp.layers.TransformerDecoder`, junto con la secuencia objetivo hasta ahora (palabras objetivo de 0 a N). El `keras_nlp.layers.TransformerDecoder` buscará predecir las próximas palabras en la secuencia objetivo (N+1 en adelante).

Un detalle clave que hace esto posible es el enmascaramiento causal. El `keras_nlp.layers.TransformerDecoder` ve toda la secuencia de una vez, por lo que debemos asegurarnos de que solo utilice información de los tokens objetivo de 0 a N al predecir el token N+1 (de lo contrario, podría utilizar información del futuro, lo que resultaría en un modelo que no se puede utilizar en el momento de la inferencia). El enmascaramiento causal está habilitado de forma predeterminada en `keras_nlp.layers.TransformerDecoder`.

También necesitamos enmascarar los tokens de relleno (`"[PAD]"`). Para ello, podemos configurar el argumento `mask_zero` de la capa `keras_nlp.layers.TokenAndPositionEmbedding` en True. Esto se propagará a todas las capas subsiguientes.

In [15]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=SPA_VOCAB_SIZE,  # Usar el tamaño del vocabulario español
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,  # Usar el tamaño del vocabulario inglés
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(ENG_VOCAB_SIZE, activation="softmax")(x)  # Salida en inglés
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

## Entrenando nuestro modelo

Utilizaremos la precisión como una forma rápida de monitorear el progreso del entrenamiento en los datos de validación. Es importante destacar que la traducción automática generalmente utiliza puntajes BLEU, así como otras métricas, en lugar de la precisión. Sin embargo, para utilizar métricas como ROUGE, BLEU, etc., deberíamos decodificar las probabilidades y generar el texto. La generación de texto es computacionalmente costosa y realizarla durante el entrenamiento no es recomendable.

In [16]:
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 token_and_position_embeddi  (None, None, 256)            3850240   ['encoder_inputs[0][0]']      
 ng (TokenAndPositionEmbedd                                                                       
 ing)                                                                                             
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                  

El modelo Transformer tiene tres capas principales:

1. `token_and_position_embedding`: Esta capa toma las secuencias de entrada del encoder y del decoder y las transforma en representaciones vectoriales con incrustaciones de tokens y posiciones.

2. `transformer_encoder`: Esta capa es la parte del codificador del modelo Transformer. Toma las representaciones vectoriales del `token_and_position_embedding` y las procesa a través de capas de atención y redes neuronales completamente conectadas.

3. `model_1`: Esta capa es la parte del decodificador del modelo Transformer. Toma las secuencias de entrada del decoder y las procesa a través de capas de atención y redes neuronales completamente conectadas. Su salida es la secuencia de palabras en el idioma de destino.

El modelo tiene un total de 14,449,304 parámetros entrenables y su tamaño en memoria es de aproximadamente 55.12 MB.

In [17]:
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

En esta parte del código, se compila el modelo Transformer utilizando el optimizador "rmsprop" como método de optimización. Además, se utiliza la función de pérdida "sparse_categorical_crossentropy" que es comúnmente utilizada para problemas de clasificación con múltiples clases, como la generación de texto. Como métrica de evaluación durante el entrenamiento, se utiliza "accuracy" para monitorear la precisión del modelo en los datos de validación.

In [18]:
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 129s 91ms/step - loss: 0.9954 - accuracy: 0.8480 - val_loss: 0.7095 - val_accuracy: 0.8814
Epoch 2/30
1302/1302 [==============================] - 97s 75ms/step - loss: 0.6209 - accuracy: 0.8959 - val_loss: 0.5092 - val_accuracy: 0.9111
Epoch 3/30
1302/1302 [==============================] - 97s 74ms/step - loss: 0.4898 - accuracy: 0.9140 - val_loss: 0.4395 - val_accuracy: 0.9209
Epoch 4/30
1302/1302 [==============================] - 107s 82ms/step - loss: 0.4247 - accuracy: 0.9229 - val_loss: 0.4045 - val_accuracy: 0.9261
Epoch 5/30
1302/1302 [==============================] - 97s 75ms/step - loss: 0.3836 - accuracy: 0.9288 - val_loss: 0.3908 - val_accuracy: 0.9289
Epoch 6/30
1302/1302 [==============================] - 107s 82ms/step - loss: 0.3538 - accuracy: 0.9334 - val_loss: 0.3803 - val_accuracy: 0.9305
Epoch 7/30
1302/1302 [==============================] - 97s 75ms/step - loss: 0.3302 - accuracy: 0.9369 - val_loss: 0.381

El modelo ha sido entrenado durante 30 épocas. Durante el entrenamiento, la pérdida (loss) en los datos de entrenamiento disminuyó gradualmente, lo que indica que el modelo está aprendiendo y mejorando su capacidad para realizar traducciones. Además, la precisión (accuracy) en los datos de entrenamiento también aumentó, lo que indica que el modelo está haciendo predicciones más precisas en cada época.

En los datos de validación, observamos un patrón similar de disminución de la pérdida y aumento de la precisión, lo que sugiere que el modelo generaliza bien a datos no vistos.

Sin embargo, para determinar si el entrenamiento es bueno o no, sería necesario evaluar el modelo en datos de prueba independientes y medir métricas de evaluación como BLEU o ROUGE, que son más adecuadas para la evaluación de traducción automática. Estos resultados proporcionarían una evaluación más precisa del rendimiento del modelo en tareas de traducción de idiomas.

## Decodificación de oraciones de prueba (análisis cualitativo)

Finalmente, vamos a demostrar cómo traducir nuevas oraciones en inglés. Simplemente alimentamos al modelo la oración en inglés tokenizada, así como el token objetivo `"[START]"`. El modelo genera probabilidades del siguiente token. Luego, generamos repetidamente el siguiente token condicionado a los tokens generados hasta ahora, hasta que llegamos al token `"[END]"`.

Para la decodificación, utilizaremos el módulo `keras_nlp.samplers` de KerasNLP. La Decodificación Greedy es un método de decodificación de texto que genera el token siguiente más probable en cada paso de tiempo, es decir, el token con la probabilidad más alta.

In [19]:
def decode_sequences(input_sentences):
    batch_size = 1

    # Tokenizar la entrada del codificador (en español).
    encoder_input_tokens = spa_tokenizer(input_sentences).to_tensor()

    # Si la longitud de la secuencia es menor que MAX_SEQUENCE_LENGTH, aplicar padding.
    sequence_length = tf.shape(encoder_input_tokens)[1]
    if sequence_length < MAX_SEQUENCE_LENGTH:
        pads = tf.fill([batch_size, MAX_SEQUENCE_LENGTH - sequence_length], 0)
        encoder_input_tokens = tf.concat([encoder_input_tokens, pads], 1)

    # Definir una función para la decodificación.
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        return logits, None, cache

    # Construir un prompt con un token de inicio y tokens de padding.
    length = 40
    start = tf.fill([batch_size, 1], eng_tokenizer.token_to_id("[START]"))
    pad = tf.fill([batch_size, length - 1], eng_tokenizer.token_to_id("[PAD]"))
    prompt = tf.concat([start, pad], axis=-1)

    generated_tokens = keras_nlp.samplers.GreedySampler()(
        next,
        prompt,
        end_token_id=eng_tokenizer.token_to_id("[END]"),
        index=1,
    )
    generated_sentences = eng_tokenizer.detokenize(generated_tokens)
    return generated_sentences

test_spa_texts = [pair[0] for pair in test_pairs]
for i in range(10):
    input_sentence = random.choice(test_spa_texts)
    translated = decode_sequences([input_sentence])
    translated = translated.numpy()[0].decode("utf-8")
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    print(f"** Example {i} **")
    print(input_sentence)
    print(translated)
    print()

** Example 0 **
me pareció muy divertido.
i thought it was very fun .

** Example 1 **
yo estoy siendo buena con ustedes esta mañana.
i ' m being good to you this morning .

** Example 2 **
a la mañana siguiente el muñeco de nieve estaba completamente derretido.
the morning after the morningoco was completely melregr .

** Example 3 **
quería comprar el libro.
i wanted to get the book .

** Example 4 **
ese movimiento fue un gran error.
that motar was a big mistake .

** Example 5 **
todavía no se ha leído el libro.
not haven ' t read the book yet .

** Example 6 **
le di tres libros de texto a cambio de su ayuda.
he is three designed books of his help .

** Example 7 **
tom trabaja en arqueología.
tom works intrchology .

** Example 8 **
es francés.
french is .

** Example 9 **
estaría bien tener una de esas, ¿a que sí?
you would be good at those who do to ?



## Evaluación de nuestro proyecto (análisis cuantitativo)

Existen muchas métricas que se utilizan para tareas de generación de texto. Aquí, para evaluar las traducciones generadas por nuestro proyecto, calcularemos los puntajes ROUGE-1 y ROUGE-2. Básicamente, ROUGE-N es un puntaje basado en el número de n-gramas comunes entre el texto de referencia y el texto generado. ROUGE-1 y ROUGE-2 utilizan el número de unigramas y bigramas comunes, respectivamente.

Calcularemos el puntaje en base a 30 muestras de prueba (ya que la decodificación es un proceso costoso).

In [20]:
rouge_1 = keras_nlp.metrics.RougeN(order=1)
rouge_2 = keras_nlp.metrics.RougeN(order=2)

for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    translated_sentence = decode_sequences([input_sentence])
    translated_sentence = translated_sentence.numpy()[0].decode("utf-8")
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )

    # Evaluar con ROUGE-1 y ROUGE-2
    rouge_1.update_state([reference_sentence], [translated_sentence])
    rouge_2.update_state([reference_sentence], [translated_sentence])

# Obtener el diccionario completo de puntuaciones ROUGE
rouge_1_scores = rouge_1.result()
rouge_2_scores = rouge_2.result()

# Imprimir el diccionario completo
print("ROUGE-1 Scores: ", rouge_1_scores)
print("ROUGE-2 Scores: ", rouge_2_scores)

ROUGE-1 Scores:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.6953174>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.6882178>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.6867349>}
ROUGE-2 Scores:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.51784396>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.5083622>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.5093947>}


Después de 30 épocas, los puntajes son los siguientes:

|               | **ROUGE-1** | **ROUGE-2** |
|:-------------:|:-----------:|:-----------:|
| **Precision** |    0.6953174    |    0.51784396    |
|   **Recall**  |    0.6882178    |    0.5083622    |
|  **F1 Score** |    0.6867349    |    0.5093947    |

Estos puntajes son una medida de la calidad de las traducciones generadas por nuestro modelo. En general, los puntajes indican lo siguiente:

- ROUGE-1 Precision: El 69.53% de las palabras en las traducciones generadas coinciden con las palabras en las referencias de manera precisa.
- ROUGE-1 Recall: Se logra recuperar el 68.82% de las palabras presentes en las referencias.
- ROUGE-1 F1 Score: Un equilibrio entre precisión y recall con un valor de 0.6867.

- ROUGE-2 Precision: El 51.78% de los bigramas (pares de palabras) en las traducciones coinciden con los bigramas en las referencias de manera precisa.
- ROUGE-2 Recall: Se recupera el 50.84% de los bigramas presentes en las referencias.
- ROUGE-2 F1 Score: Un equilibrio entre precisión y recall para bigramas con un valor de 0.5094.

En resumen, los puntajes indican que nuestro modelo logra generar traducciones con una precisión razonable, capturando una parte significativa del contenido de las referencias. Sin embargo, aún hay margen de mejora, especialmente en el caso de los bigramas (ROUGE-2), donde la precisión y el recall son más bajos en comparación con los unigramas (ROUGE-1). Esto sugiere que el modelo podría beneficiarse de mejoras en la coherencia y la fluidez de las traducciones de frases más largas.

# Comparación entre Transformer Secuencia a Secuencia y KerasNLP

Al comparar los resultados obtenidos en los dos proyectos, uno utilizando un enfoque tradicional de Transformer secuencia a secuencia y el otro empleando KerasNLP, podemos observar varias diferencias y similitudes en términos de rendimiento, calidad de las traducciones y las características inherentes de cada enfoque:

1. **Calidad de la Traducción**:
   - **Transformer Secuencia a Secuencia Tradicional**: Las traducciones generadas por este modelo muestran una comprensión básica de la estructura del lenguaje, pero con ciertas limitaciones en términos de precisión y fluidez. Algunas frases son traducidas de manera coherente, mientras que otras presentan errores gramaticales o de interpretación.
   - **Usando KerasNLP**: Las traducciones producidas con KerasNLP parecen ser ligeramente más fluidas y coherentes. Aunque también hay errores, especialmente en frases más complejas, el grado de naturalidad en las traducciones es notable.

2. **Manejo de Vocabulario y Estructuras Gramaticales**:
   - En el modelo de Transformer secuencia a secuencia tradicional, hay casos en los que el modelo lucha con estructuras gramaticales complejas o vocabulario específico.
   - Con KerasNLP, parece haber un mejor manejo de la diversidad lingüística, posiblemente debido a una tokenización más eficiente y un manejo más robusto del vocabulario.

3. **Coherencia y Contexto**:
   - Ambos modelos enfrentan desafíos en mantener la coherencia y capturar el contexto completo de las oraciones, pero el uso de KerasNLP puede ofrecer una ligera ventaja en términos de coherencia contextual, probablemente debido a su enfoque en las capas Transformer optimizadas.

4. **Facilidad de Implementación y Flexibilidad**:
   - KerasNLP ofrece una implementación más sencilla y modular, lo que puede acelerar el desarrollo y permitir una mayor experimentación con diferentes configuraciones de capas y parámetros.

5. **Generalización y Adaptabilidad**:
   - Mientras que el enfoque tradicional de Transformer secuencia a secuencia proporciona una base sólida, KerasNLP, con su enfoque modular y optimizado, puede ofrecer una mejor adaptabilidad a diferentes tipos de textos y tareas de traducción.

En conclusión, aunque ambos enfoques tienen sus méritos y limitaciones, el uso de KerasNLP puede ofrecer ventajas en términos de facilidad de implementación, manejo de vocabulario y coherencia en las traducciones. Sin embargo, la elección entre estos enfoques dependerá de los objetivos específicos del proyecto, la complejidad de las tareas y las preferencias en cuanto a personalización y control sobre el modelo.